
requirment:
python==3.6.5
flask

# load sample data

In [1]:
import importlib
import featurebinning as fb
importlib.reload(fb)

import pandas as pd

In [2]:
data1 = pd.read_csv("./data_sample.csv")
display(data1.head(2))
print(data1.shape)

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var40,var41,var42,var43,var44,var45,var46,var47,var48,target
0,1.5,1.5,600.0,3.0,1.0,2.2,2.666667,4.0,6.0,200.0,...,2.3061,0.0,2.000000,0.0,5.5,500.0,3.6,2.5,6000.0,0
1,0.5,1.5,200.0,NaN,1.0,NaN,50.000000,0.0,16.0,-99998.0,...,69.1830,-10000000.0,1.111111,0.0,22.5,100.0,NaN,0.5,8000.0,0


(20000, 50)


In [3]:
wt = fb.WideTable(data1)
print(wt.label_mapping)
wt.label_mapping.update({"var0":'变量0',"var2":'变量2'})
print(wt.label_mapping)

bin cache file found at d:\zhangyin\project\local_tools\feature_binning\bin_info.json
{}
{'var0': '变量0', 'var2': '变量2'}


In [4]:
wt.refs= [data1.sample(n=10000)]

In [5]:
wt.run_app()

server running on 127.0.0.1:50000
 * Serving Flask app "binning_server" (lazy loading)


<Thread(Thread-5, started daemon 32424)>

 * Environment: development
 * Debug mode: off


In [6]:
%%HTML
<iframe src="c/test?var=var7" width="90%" height="1000px" style="background-color:antiquewhite"></iframe>

In [7]:
wt.refs.append(data1.sample(n=2000))

In [5]:
df_raw = pd.read_pickle("./data_sample.pkl")
var_bin,bin_label,var_woe,woe_bin_mapping = wt.var_cut(df_raw,"var0",calculate_woe=True)
display(var_bin.head())
display(bin_label)
display(var_woe.head())
display(woe_bin_mapping)
var_bin,bin_label,var_woe,woe_bin_mapping

var0: woes set to zero for very small bins
   total  bad  good
2     10    1     9


188489    1
133130    0
118749    0
4345      0
132270    0
dtype: int8

{0: '(-99998.00200000001, 0.5]', 1: '(0.5, 20.501]', 2: '(20.501, inf]'}

188489   -0.112290
133130   -0.031534
118749   -0.031534
4345     -0.031534
132270   -0.031534
dtype: float64

{-1: 0.1428944938022283,
 0: -0.03153395573963996,
 1: -0.11229036298730305,
 2: 0.0}

In [9]:
var_bin,bin_label = wt.var_cut(df_raw,"var0",calculate_woe=False)
display(var_bin.head())
display(bin_label)

188489    1
133130    0
118749    0
4345      0
132270    0
dtype: int8

{0: '(-99998.00200000001, 0.5]', 1: '(0.5, 20.501]', 2: '(20.501, inf]'}

In [8]:
# 部署
import json

with open("./bin_info.json","r") as f:
    bin_info_dict = json.load(f)

print(",".join(bin_info_dict.keys()))






var7,var11


In [ ]:
import json
import pandas as pd
import joblib
import os
import numpy as np
from cachetools import LRUCache
from cachetools import cached
import lightgbm as lgb

# 缓存
lrucache = LRUCache(maxsize=50)
@cached(lrucache)
def get_lgbm(fname):
    print(fname)
    return joblib.load(fname)

cpath = "./" #os.path.dirname(__file__)
filename = 'a4_app_md1027_150631.pkl'
full_name = os.path.join(cpath, filename)
model_data = get_lgbm(full_name)

LGBM = lgb.Booster(train_set=lgb.Dataset(np.array([[0]])))
LGBM.model_from_string(model_data["model"])
BIN_INFO = pd.DataFrame.from_dict(json.loads(model_data["bins"]))


def encode(v,bins):
    if pd.isnull(v):
        return -1
    for i,edge in enumerate(bins[1:-1]):
        if v <= edge:
            return i
    return len(bins)-2


FEAURE_IN = "var0,var1,var2".split(",")

def model_predict(**kwargs):
    for f in FEAURE_IN:
        if f not in kwargs:
            raise Exception(f"feature {f} not in input")
    for f in kwargs:
        if f in bin_info.columns:
            kwargs[f] = encode(kwargs[f],BIN_INFO[f])

    

In [8]:
import numpy as np
import pandas as pd
import json

In [9]:
df = pd.DataFrame(np.random.randint(10,size=(100,3)),columns=["var0","var1","var2"]).astype(float)
df.iloc[0] = np.nan
df["va1"] = df["var1"].astype(str)+ '_s'

In [10]:
d1 = df.describe()
d1

,var0,var1,var2
count,99.000000,99.000000,99.000000
mean,4.545455,4.474747,4.858586
std,2.959843,2.879709,2.751660
min,0.000000,0.000000,0.000000
25%,2.000000,2.000000,3.000000
50%,4.000000,4.000000,5.000000
75%,7.500000,7.000000,7.000000
max,9.000000,9.000000,9.000000


In [11]:
d2 = df.describe(include="all")
d2

,var0,var1,var2,va1
count,99.000000,99.000000,99.000000,100
unique,NaN,NaN,NaN,11
top,NaN,NaN,NaN,1.0_s
freq,NaN,NaN,NaN,13
mean,4.545455,4.474747,4.858586,NaN
std,2.959843,2.879709,2.751660,NaN
min,0.000000,0.000000,0.000000,NaN
25%,2.000000,2.000000,3.000000,NaN
50%,4.000000,4.000000,5.000000,NaN
75%,7.500000,7.000000,7.000000,NaN


In [12]:
display(d2.to_html(na_rep='&nbsp-&nbsp',))

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>var0</th>\n      <th>var1</th>\n      <th>var2</th>\n      <th>va1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>count</th>\n      <td>99.000000</td>\n      <td>99.000000</td>\n      <td>99.000000</td>\n      <td>100</td>\n    </tr>\n    <tr>\n      <th>unique</th>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>11</td>\n    </tr>\n    <tr>\n      <th>top</th>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>1.0_s</td>\n    </tr>\n    <tr>\n      <th>freq</th>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>&amp;nbsp-&amp;nbsp</td>\n      <td>13</td>\n    </tr>\n    <tr>\n      <th>mean</th>\n      <td>4.545455</td>\n      <td>4.474747</td>\n      <td>4.858586</td>\n      <td>&amp;nbsp-&

In [13]:
from collections import defaultdict
d2.to_html(float_format="%.2f")

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>var0</th>\n      <th>var1</th>\n      <th>var2</th>\n      <th>va1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>count</th>\n      <td>99.00</td>\n      <td>99.00</td>\n      <td>99.00</td>\n      <td>100</td>\n    </tr>\n    <tr>\n      <th>unique</th>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>11</td>\n    </tr>\n    <tr>\n      <th>top</th>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>1.0_s</td>\n    </tr>\n    <tr>\n      <th>freq</th>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>13</td>\n    </tr>\n    <tr>\n      <th>mean</th>\n      <td>4.55</td>\n      <td>4.47</td>\n      <td>4.86</td>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>std</th>\n      <td>2.96</td>\n      <td>2.88</td>\n      <td>2.75</td>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>min</th>\n    

In [18]:
d2.style.format(na_rep='')

TypeError: format() got an unexpected keyword argument 'na_rep'

In [20]:
d2.style.

Type:        property
String form: <property object at 0x000002C5C9AFBE08>
Docstring:  
Property returning a Styler object containing methods for
building a styled HTML representation fo the DataFrame.

See Also
--------
pandas.io.formats.style.Styler
